In [14]:
%config IPCompleter.greedy=True

### Tensors

In [74]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

x=torch.tensor([[1,1],[-1,-1]])
print(x)
y=torch.tensor([[0,0],[1,1]])
print(y)
print(torch.add(x,y))
print(y.add_(x))
x0 = x.view(4)
print(x0)
x=torch.empty(5,3)
z = torch.randn(4,4,4)
z0 = z.view(-1,8,2)
print(z0.size())
print(torch.cuda.is_available())

tensor([[ 1,  1],
        [-1, -1]])
tensor([[0, 0],
        [1, 1]])
tensor([[1, 1],
        [0, 0]])
tensor([[1, 1],
        [0, 0]])
tensor([ 1,  1, -1, -1])
torch.Size([4, 8, 2])
False


###  AutoGrad

In [42]:
x = torch.rand(2,2,requires_grad=True)
#print(x)
y = x + 2
print(y.grad_fn)
z = y * y * 3
z_mean = z.mean()
print(z)
print(z_mean)
z_mean.backward()
print(x.grad)

tensor([[14.7668, 20.4317],
        [17.1685, 23.1552]], grad_fn=<MulBackward0>)
tensor(18.8806, grad_fn=<MeanBackward1>)
tensor([[3.3279, 3.9146],
        [3.5884, 4.1673]])


In [43]:
x = torch.randn(3,3,requires_grad=True)
print(x)

tensor([[-0.7367, -1.2929,  0.1704],
        [-0.4890, -0.2523,  0.0469],
        [-0.2459,  0.2969, -2.6363]], requires_grad=True)


### MNIST TASK

In [150]:
from torch.autograd import Variable
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(784,15)
        self.fc2 = nn.Linear(15,10)
        self.fc3 = nn.Linear(10,10)
        
    def forward(self,x):
        x=torch.sigmoid(self.fc1(x))
        x=torch.sigmoid(self.fc2(x))
        x=torch.sigmoid(self.fc3(x))
        return x
    
    
    def train(self, data_loader, epochs=13, bs=64, criterion=nn.MSELoss()):
        opt = optim.SGD(self.parameters(), lr=0.01)
        for epoch in range(epochs):
            print(epoch)
            for xb,yb in data_loader:
                pred = self(xb)
                print(pred)
                print(yb)
                loss = criterion(pred, yb)
                print("loss=", loss)
                ###
                loss.backward()
                opt.step()
                opt.zero_grad() 
                
            
            
            


In [154]:
from pathlib import Path
import requests

# Extracting the data
DATA_PATH = Path("data")
PATH = DATA_PATH / "mnist"

PATH.mkdir(parents=True, exist_ok=True)

URL = "http://deeplearning.net/data/mnist/"
FILENAME = "mnist.pkl.gz"

if not (PATH / FILENAME).exists():
        content = requests.get(URL + FILENAME).content
        (PATH / FILENAME).open("wb").write(content)
import pickle
import gzip

with gzip.open((PATH / FILENAME).as_posix(), "rb") as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")
        
# Creating tensors instead of numpy arrays
x_train, y_train, x_valid, y_valid = map(
    torch.tensor, (x_train, y_train, x_valid, y_valid))
print(y_train)
print(y_valid)
print(y_valid.size())
# Creating Dataset
tr_data_set = TensorDataset(x_train, y_train)
tr_data_loader = DataLoader(tr_data_set, batch_size=64)



tensor([5, 0, 4,  ..., 8, 4, 8])
tensor([3, 8, 6,  ..., 5, 6, 8])
torch.Size([10000])


In [152]:
net=Net()
#print(loss_func(net(x_train),y_train))

net.train(tr_data_loader)

0
tensor([[0.5385, 0.3286, 0.6535, 0.4355, 0.2795, 0.3338, 0.3331, 0.5587, 0.4908,
         0.6062],
        [0.5385, 0.3290, 0.6549, 0.4339, 0.2793, 0.3330, 0.3310, 0.5608, 0.4910,
         0.6058],
        [0.5386, 0.3295, 0.6545, 0.4334, 0.2796, 0.3342, 0.3309, 0.5607, 0.4910,
         0.6049],
        [0.5390, 0.3291, 0.6547, 0.4338, 0.2797, 0.3338, 0.3321, 0.5602, 0.4909,
         0.6052],
        [0.5393, 0.3291, 0.6561, 0.4333, 0.2794, 0.3332, 0.3313, 0.5612, 0.4905,
         0.6054],
        [0.5380, 0.3290, 0.6545, 0.4336, 0.2791, 0.3334, 0.3312, 0.5609, 0.4908,
         0.6049],
        [0.5392, 0.3302, 0.6544, 0.4337, 0.2805, 0.3339, 0.3314, 0.5596, 0.4911,
         0.6045],
        [0.5375, 0.3282, 0.6557, 0.4343, 0.2785, 0.3334, 0.3316, 0.5610, 0.4912,
         0.6063],
        [0.5387, 0.3300, 0.6540, 0.4345, 0.2801, 0.3337, 0.3320, 0.5594, 0.4907,
         0.6048],
        [0.5383, 0.3296, 0.6545, 0.4324, 0.2788, 0.3331, 0.3299, 0.5630, 0.4905,
         0.6042],
        

RuntimeError: The size of tensor a (10) must match the size of tensor b (64) at non-singleton dimension 1